# Poisson is our savior

https://www.searchenginejournal.com/python-seo-forecasting/420237/#close


# Points to Remember 
- Quantitative forecasting
- Reorder point = (# units used daily x # days lead time) + # units safety stock


```Sales velocity (SV) = [(# of leads) x (average deal value) x (% conversion rate)] / (sales cycle length)```
For example, imagine a scenario where your company has 20 opportunities to sell a product. The percent of these opportunities that usually get turned into a sale is about 50%. The average deal size is `$5,000`, with a sales cycle length of three months. Sales velocity is `[(20 x .50 x $5,000)/90 days] = $555.56`. This number means that the product is bringing in about $556 per day in revenue.


# Inventory Forecasting Formulas
- EOQ = √2DS/H, where
    - D = Demand in units per year
    - S = Order cost per purchase
    - H = Holding cost per unit, per year
- Reorder point = (# units used daily x # days lead time) + (# units safety stock)
- Average inventory = (Beginning inventory + ending inventory) / 2
- Inventory Turnover Ratio = COGS / average inventory
    - How many times has your company sold and replenished its inventory over the last year? The inventory turnover ratio helps you see how many days it will take to sell the inventory you have on hand. A higher ratio points to strong sales.
    - COGS = sum of all direct costs of producing goods, including raw materials as well as your average inventory
- Safety stock
    - Safety stock = (Maximum number of units sold in a day X maximum lead time for stock replenishment) — (average daily usage X average lead time in days)
- Gross Margin Return on Invested Inventory (GMROI)
    - GMROI = Gross profit margin / average cost of inventory on hand
    - Retail businesses use this formula to see how well they are turning inventory into profits.


In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from pandas_profiling import ProfileReport
import seaborn as sns
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('neat.csv')
# pf = ProfileReport(df_dp,title="Profile")

# Cleaned Data by considering Booked_date has correct year
- Copied the Booked Date's year to the fiscal quarter and month

In [9]:
df.head(5)

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Quarter,Month,Year
0,SRSBU,A9K1000,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,01-01-2013,Q2,JAN,2013
1,TOASTBU,SFP10G,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,01-01-2013,Q2,JAN,2013
2,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,01-02-2013,Q3,FEB,2013
3,TOASTBU,SFP10G,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,01-02-2013,Q3,FEB,2013
4,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,01-03-2013,Q3,MAR,2013


,PLID,Booked_Qty,Quarter,Month,Year
1,201,18921,1,4,2013
3,201,12810,2,3,2013
5,201,14069,2,7,2013
8,201,36158,2,0,2013
11,201,13622,3,8,2013
...,...,...,...,...,...
9936,201,47303,0,1,2022
10184,201,111320,0,11,2022
10434,201,270600,0,10,2022
10683,201,58354,1,9,2022


In [ ]:
print(hello)

# Creating Quarter Column

In [ ]:
quarter_list = df['Fiscal Quarter']
quarter_list = np.array(quarter_list)
index = 0
for i in quarter_list:
    q_y = i.split(' ')
    # print(q_y)
    quarter_list[index] = q_y[0]
    index += 1
    # break
print(quarter_list)

['Q2' 'Q2' 'Q3' ... 'Q2' 'Q2' 'Q2']


In [ ]:
# # # CHeck if it quartereed properly or not
# for k in quarter_list:
#     if(len(k)>2):
#         print(k)

In [ ]:
df['Quarter'] = quarter_list

In [ ]:
df

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Quarter,Month,Year
0,SRSBU,A9K1000,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,01-01-2013,Q2,1,2013
1,TOASTBU,SFP10G,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,01-01-2013,Q2,1,2013
2,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,01-02-2013,Q3,2,2013
3,TOASTBU,SFP10G,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,01-02-2013,Q3,2,2013
4,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,01-03-2013,Q3,3,2013
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9120AX,C9120AXE-B,Q2 FY2022,DEC FY2022,19845,01-12-2022,Q2,12,2022
10992,RTOBU,C9120AX,C9120AXI-B,Q2 FY2022,DEC FY2022,88838,01-12-2022,Q2,12,2022
10993,RTOBU,C9130AX,C9130AXE-B,Q2 FY2022,DEC FY2022,3214,01-12-2022,Q2,12,2022
10994,RTOBU,C9130AX,C9130AXI-B,Q2 FY2022,DEC FY2022,54344,01-12-2022,Q2,12,2022


# Creating Month Column

In [ ]:
month_list = df['Fiscal Month']
month_list = np.array(month_list)
index = 0
# months = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
for i in month_list:
    q_y = i.split(' ')[0]
    # print(q_y)
    month_list[index] = q_y;
    index += 1
    # break
print(month_list)

['JAN' 'JAN' 'FEB' ... 'DEC' 'DEC' 'DEC']


In [ ]:
# CHECK if Month is monthed properly
# for k in month_list:
#     if(len(k)>3):
#         print(k)

In [ ]:
df['Month'] = month_list
df

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Quarter,Month,Year
0,SRSBU,A9K1000,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,01-01-2013,Q2,JAN,2013
1,TOASTBU,SFP10G,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,01-01-2013,Q2,JAN,2013
2,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,01-02-2013,Q3,FEB,2013
3,TOASTBU,SFP10G,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,01-02-2013,Q3,FEB,2013
4,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,01-03-2013,Q3,MAR,2013
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9120AX,C9120AXE-B,Q2 FY2022,DEC FY2022,19845,01-12-2022,Q2,DEC,2022
10992,RTOBU,C9120AX,C9120AXI-B,Q2 FY2022,DEC FY2022,88838,01-12-2022,Q2,DEC,2022
10993,RTOBU,C9130AX,C9130AXE-B,Q2 FY2022,DEC FY2022,3214,01-12-2022,Q2,DEC,2022
10994,RTOBU,C9130AX,C9130AXI-B,Q2 FY2022,DEC FY2022,54344,01-12-2022,Q2,DEC,2022


# Creating Year Column

In [ ]:
year_list = df['Fiscal Month']
year_list = np.array(year_list)
index = 0
for i in year_list:
    q_y = i.split(' ')
    # print(q_y)
    # print(q_y[1])
    year_list[index] = int(q_y[1].split('Y')[1])
    # year_list[index] = q_y[1]
    index += 1
    # break
print(year_list)

[2013 2013 2013 ... 2022 2022 2022]


In [ ]:
# CHECK if Month is monthed properly
for k in year_list:
    if(k<2002):
        print(k)

In [ ]:
df['Year'] = year_list
df

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Quarter,Month,Year
0,SRSBU,A9K1000,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,01-01-2013,Q2,JAN,2013
1,TOASTBU,SFP10G,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,01-01-2013,Q2,JAN,2013
2,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,01-02-2013,Q3,FEB,2013
3,TOASTBU,SFP10G,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,01-02-2013,Q3,FEB,2013
4,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,01-03-2013,Q3,MAR,2013
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9120AX,C9120AXE-B,Q2 FY2022,DEC FY2022,19845,01-12-2022,Q2,DEC,2022
10992,RTOBU,C9120AX,C9120AXI-B,Q2 FY2022,DEC FY2022,88838,01-12-2022,Q2,DEC,2022
10993,RTOBU,C9130AX,C9130AXE-B,Q2 FY2022,DEC FY2022,3214,01-12-2022,Q2,DEC,2022
10994,RTOBU,C9130AX,C9130AXI-B,Q2 FY2022,DEC FY2022,54344,01-12-2022,Q2,DEC,2022


In [ ]:
df.to_csv('neat.csv',index=False)

# Trying to find Trends and patterns in **Data**

In [ ]:
df_dp = df.drop(['Business Unit','Product Family','Fiscal Quarter','Fiscal Month','Booking_Date'], axis=1)
df_dp

,PLID,Booked_Qty,Quarter,Month,Year
0,A9K1000-ESP100,63,Q2,JAN,2013
1,SFP-10G-LR,18921,Q2,JAN,2013
2,A9K1000-ESP100,31,Q3,FEB,2013
3,SFP-10G-LR,12810,Q3,FEB,2013
4,A9K1000-ESP100,61,Q3,MAR,2013
...,...,...,...,...,...
10991,C9120AXE-B,19845,Q2,DEC,2022
10992,C9120AXI-B,88838,Q2,DEC,2022
10993,C9130AXE-B,3214,Q2,DEC,2022
10994,C9130AXI-B,54344,Q2,DEC,2022


In [10]:
df_dp = df.drop(['Business Unit','Product Family','Fiscal Quarter','Fiscal Month','Booking_Date'], axis=1)
le = LabelEncoder()
df_dp['PLID'] = le.fit_transform(df_dp['PLID'])
product_dict = dict(zip(le.classes_, le.transform(le.classes_)))
print(product_dict)
# df_dp['Quarter'] = le.fit_transform(df_dp['Quarter'])
# quarter_dict = dict(zip(le.classes_, le.transform(le.classes_)))
# print(quarter_dict)
# df_dp['Month'] = le.fit_transform(df_dp['Month'])
# month_dict = dict(zip(le.classes_, le.transform(le.classes_)))
# print(month_dict)

{'8201-SYS': 0, '88-LC0-36FH': 1, '8800-LC-36FH': 2, '8800-LC-48H': 3, '8808-FC': 4, '8808-SYS': 5, '8812-SYS': 6, '8818-SYS': 7, 'A900-IMA-8Z': 8, 'A99-32X100GE-X-SE': 9, 'A9K-24X10GE-1G-TR': 10, 'A9K-8X100GE-TR': 11, 'A9K-920-12CZ-A': 12, 'A9K-920-12SZ-IM': 13, 'A9K-920-4SZ-D': 14, 'A9K-9901': 15, 'A9K-9903': 16, 'A9K-RSP5-SE': 17, 'A9K-RSP5-TR': 18, 'A9K-RSP880-SE': 19, 'A9K-RSP880-TR': 20, 'A9K1000-ESP100': 21, 'A9K1001-HX': 22, 'A9K1001-X': 23, 'A9K1002-HX': 24, 'C1000-24T-4G-L': 25, 'C1000-48T-4G-L': 26, 'C1111-4P': 27, 'C1111-8P': 28, 'C1111-8PLTExA': 29, 'C1116-4P': 30, 'C1121-8P': 31, 'C8200-1N-4T': 32, 'C8200L-1N-4T': 33, 'C8300-1N1S-6T': 34, 'C8300-2N2S-4T2X': 35, 'C8300-2N2S-6T': 36, 'C8500-12X': 37, 'C8500-12X4QC': 38, 'C8500L-8S4X': 39, 'C9105AXI-B': 40, 'C9105AXW-B': 41, 'C9115AXE-B': 42, 'C9115AXI-B': 43, 'C9120AXE-B': 44, 'C9120AXI-B': 45, 'C9130AXE-B': 46, 'C9130AXI-B': 47, 'C9200-24T-E': 48, 'C9200-48P-A': 49, 'C9200-48P-E': 50, 'C9200-48PXG-E': 51, 'C9200-48T-E': 52

In [ ]:
X = np.array(df_dp[['PLID','Quarter','Month','Year']])
y = np.array(df_dp['Booked_Qty'])

In [ ]:
X

array([[21, 'Q2', 'JAN', 2013],
       [201, 'Q2', 'JAN', 2013],
       [21, 'Q3', 'FEB', 2013],
       ...,
       [46, 'Q2', 'DEC', 2022],
       [47, 'Q2', 'DEC', 2022],
       [92, 'Q2', 'DEC', 2022]], dtype=object)

In [ ]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(hello)

# Mulitple Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = regressor.predict(X_test)
y_pred
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

print(regressor.score(X_test, y_test))

[[ 7756.8     55.  ]
 [ 7653.82    41.  ]
 [ 7072.44 14859.  ]
 ...
 [ 7384.07 10619.  ]
 [ 6623.78    24.  ]
 [ 7759.01   208.  ]]
-0.006791124495417744


# Polynomial Regression

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=10, random_state=0)
regressor.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10, random_state=0)

In [ ]:
X

array([[ 21,   1,   4],
       [201,   1,   4],
       [ 21,   2,   3],
       ...,
       [ 46,   1,   2],
       [ 47,   1,   2],
       [ 92,   1,   2]])

In [ ]:
predictions = regressor.predict(X_test)
for p in range(len(predictions)):
    print(predictions[p], X_test[p], y_test[p])

5032.755952380952 [135   3   5] 55
80.10999999999999 [20  3  6] 41
43039.59102564103 [198   1   4] 14859
1671.5426050420167 [107   0  11] 1826
97.64333333333335 [72  3  6] 387
4026.6538888888886 [27  0 10] 16091
1801.7166666666667 [139   1   2] 811
517.6899999999999 [154   1   2] 12
10873.08380952381 [104   2   0] 35191
8.249999999999998 [186   2   3] 0
2546.6625 [31  1  4] 39
11146.966666666667 [40  1  2] 1659
630.4599999999999 [147   2   7] 1552
50.44428571428572 [180   0   1] 36
2668.658095238095 [242   0  10] 2057
93794.0148809524 [97  2  0] 118397
608.86 [206   3   5] 14
102.71666666666665 [3 3 6] 8
59.66273809523809 [181   1   4] 193
682.0333333333333 [74  1  2] 1239
4264.15 [79  0 10] 8223
298.83333333333337 [72  2  3] 44
2.0 [6 1 4] 9
138.88416666666666 [157   3   8] 162
2699.8369047619044 [70  2  3] 2223
96.5842857142857 [181   3   5] 174
287.9166666666667 [145   0  10] 32
535.6833333333333 [67  0  1] 127
999.7595238095239 [80  0 11] 1866
400.07142857142856 [124   1   2] 402
2

In [ ]:
regressor.score(X_test,y_test)

0.6307142352399733

# Neural Networks

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_fs = sc.fit_transform(X_train)
X_test_fs = sc.transform(X_test)

In [ ]:
X_train_fs

array([[-0.01, -0.4 , -1.02],
       [ 0.53, -0.4 , -1.02],
       [ 1.83, -0.4 , -1.02],
       ...,
       [-0.23,  1.38,  0.71],
       [-1.45, -0.4 ,  1.  ],
       [ 0.22,  0.49,  0.42]])

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))
ann.add(tf.keras.layers.Dense(units=5, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='softmax'))

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
ann.fit(X_train_fs, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
310/310 [==============================] - 2s 3ms/step - loss: -14863.7930 - accuracy: 0.0083
Epoch 2/100
310/310 [==============================] - 1s 3ms/step - loss: -84568.0312 - accuracy: 0.0083
Epoch 3/100
310/310 [==============================] - 1s 2ms/step - loss: -278505.0938 - accuracy: 0.0083
Epoch 4/100
310/310 [==============================] - 1s 2ms/step - loss: -673549.6250 - accuracy: 0.0083
Epoch 5/100
310/310 [==============================] - 1s 2ms/step - loss: -1433260.7500 - accuracy: 0.0083
Epoch 6/100
310/310 [==============================] - 1s 2ms/step - loss: -2631340.0000 - accuracy: 0.0083
Epoch 7/100
310/310 [==============================] - 1s 2ms/step - loss: -4291159.5000 - accuracy: 0.0083
Epoch 8/100
310/310 [==============================] - 1s 2ms/step - loss: -6453204.5000 - accuracy: 0.0083
Epoch 9/100
310/310 [==============================] - 1s 2ms/step - loss: -9130434.0000 - accuracy: 0.0083
Epoch 10/100
310/310 [============

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

35/35 [==============================] - 0s 1ms/step
[[    1    55]
 [    1    41]
 [    1 14859]
 ...
 [    1 10619]
 [    1    24]
 [    1   208]]


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 0 20  0 ...  0  0  0]
 [ 0  9  0 ...  0  0  0]
 [ 0  8  0 ...  0  0  0]
 ...
 [ 0  1  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]]


0.008181818181818182

# XG Boost

In [ ]:
df_dp

,PLID,Booked_Qty,Quarter,Month,Year
0,21,63,1,4,2013
1,201,18921,1,4,2013
2,21,31,2,3,2013
3,201,12810,2,3,2013
4,21,61,2,7,2013
...,...,...,...,...,...
10991,44,19845,1,2,2022
10992,45,88838,1,2,2022
10993,46,3214,1,2,2022
10994,47,54344,1,2,2022


In [ ]:
from xgboost import XGBRegressor
classifier = XGBRegressor()
classifier.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
print(y_pred)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
# accuracy_score(y_test, y_pred)

[ 4132.4    168.5  40757.23 ...  7673.66  5805.01   406.88]


In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 64.06 %
Standard Deviation: 10.00 %


# CatBoost

In [ ]:
!pip install catboost

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from catboost import CatBoostClassifier
classifier = CatBoostClassifier()
classifier.fit(X_train, y_train)

Learning rate set to 0.065557
0:	learn: 3.2030682	total: 51.5ms	remaining: 51.4s
1:	learn: 3.1866949	total: 53.6ms	remaining: 26.7s
2:	learn: 3.1807678	total: 55.9ms	remaining: 18.6s
3:	learn: 3.1618180	total: 58.1ms	remaining: 14.5s
4:	learn: 3.1430401	total: 61.7ms	remaining: 12.3s
5:	learn: 3.1242540	total: 65ms	remaining: 10.8s
6:	learn: 3.1061163	total: 68.5ms	remaining: 9.72s
7:	learn: 3.0874646	total: 71.8ms	remaining: 8.9s
8:	learn: 3.0724772	total: 74.3ms	remaining: 8.18s
9:	learn: 3.0573428	total: 76.6ms	remaining: 7.58s
10:	learn: 3.0414467	total: 78.9ms	remaining: 7.09s
11:	learn: 3.0238476	total: 81.3ms	remaining: 6.7s
12:	learn: 3.0111449	total: 83.8ms	remaining: 6.37s
13:	learn: 2.9945807	total: 86.2ms	remaining: 6.07s
14:	learn: 2.9780445	total: 88.5ms	remaining: 5.81s
15:	learn: 2.9651261	total: 89.9ms	remaining: 5.53s
16:	learn: 2.9485092	total: 92ms	remaining: 5.32s
17:	learn: 2.9342534	total: 94.2ms	remaining: 5.14s
18:	learn: 2.9163433	total: 96.4ms	remaining: 4.97

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 1 0]]


0.0

In [ ]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 2)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

ValueError: ignored

# Trying out all

In [38]:
!pip install fancyimpute

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 KB 12.5 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29898 sha256=d059e959c8dcd6471e8b03ca53ddd0a7081c143f92e791803623c6ee49a7a816
  Stored in directory: /root/.cache/pip/wheels/f9/fc/6a/b0406b906bce293abe23c3b6da5a72637d2d04146ef1125a0b
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11352 sha256=e171ac2abb064ce9595d47461582b7b8a82795a79a3c8d55ce4b3d29559b1c5c
  Stored in directory: /root/.cache/pip/wheels/88/c4/be/e232c750d9bc360abf9a5e2cafe0d3e08e3605d2801bb11684
Successfully built fancyimpute knnimpute


In [39]:
from fancyimpute import IterativeImputer
from sklearn.metrics import mean_squared_error

from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import ElasticNet, Lasso, RidgeCV,LinearRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb


In [ ]:
df

,Business Unit,Product Family,PLID,Fiscal Quarter,Fiscal Month,Booked_Qty,Booking_Date,Quarter,Month,Year
0,SRSBU,A9K1000,A9K1000-ESP100,Q2 FY2013,JAN FY2013,63,01-01-2013,Q2,JAN,2013
1,TOASTBU,SFP10G,SFP-10G-LR,Q2 FY2013,JAN FY2013,18921,01-01-2013,Q2,JAN,2013
2,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,FEB FY2013,31,01-02-2013,Q3,FEB,2013
3,TOASTBU,SFP10G,SFP-10G-LR,Q3 FY2013,FEB FY2013,12810,01-02-2013,Q3,FEB,2013
4,SRSBU,A9K1000,A9K1000-ESP100,Q3 FY2013,MAR FY2013,61,01-03-2013,Q3,MAR,2013
...,...,...,...,...,...,...,...,...,...,...
10991,RTOBU,C9120AX,C9120AXE-B,Q2 FY2022,DEC FY2022,19845,01-12-2022,Q2,DEC,2022
10992,RTOBU,C9120AX,C9120AXI-B,Q2 FY2022,DEC FY2022,88838,01-12-2022,Q2,DEC,2022
10993,RTOBU,C9130AX,C9130AXE-B,Q2 FY2022,DEC FY2022,3214,01-12-2022,Q2,DEC,2022
10994,RTOBU,C9130AX,C9130AXI-B,Q2 FY2022,DEC FY2022,54344,01-12-2022,Q2,DEC,2022


In [ ]:
df_dp

,PLID,Booked_Qty,Quarter,Month,Year
0,21,63,Q2,JAN,2013
1,201,18921,Q2,JAN,2013
2,21,31,Q3,FEB,2013
3,201,12810,Q3,FEB,2013
4,21,61,Q3,MAR,2013
...,...,...,...,...,...
10991,44,19845,Q2,DEC,2022
10992,45,88838,Q2,DEC,2022
10993,46,3214,Q2,DEC,2022
10994,47,54344,Q2,DEC,2022


In [ ]:
product_dict['A9K1000-ESP100']

21

In [ ]:
# Get all product data
df_Product = df_dp[df_dp['PLID'] == 44]

# Get Independent and Dependent
X = np.array(df_Product[['PLID','Quarter','Month','Year']])
y = np.array(df_Product['Booked_Qty'])

# One Hot Encoding
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-2])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


classifiers = [
    LinearRegression(),
    RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]),
    Lasso(alpha =16, random_state=100),
    ElasticNet(alpha=0.8),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    SVR(), 
    LinearSVR(), 
    xgb.XGBRegressor(),
    lgb.LGBMRegressor()
    ]

name = []
score = []
models = []
rmse = []
i = 0
for classifier in classifiers:
    classifier.fit(X_train, y_train)   
    name.append(type(classifier).__name__)
    score.append(classifier.score(X_test, y_test))
    models.append(classifier)
    rmse.append(np.sqrt(mean_squared_error(classifier.predict(X_test), y_test)))
    from sklearn.model_selection import cross_val_score

df_score = pd.DataFrame(list(zip(name,rmse, score, models)),columns=['name','rmse','score',"model"])
df_score.set_index('name',inplace=True)
df_score.sort_values(by=['score'], ascending= False,inplace=True)
df_score

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


,rmse,score,model
name,,,
DecisionTreeRegressor,3235.995571,0.942295,DecisionTreeRegressor()
GradientBoostingRegressor,4504.773459,0.888174,([DecisionTreeRegressor(criterion='friedman_ms...
XGBRegressor,5123.218569,0.855362,"XGBRegressor(base_score=None, booster=None, ca..."
RandomForestRegressor,5352.976830,0.842098,"(DecisionTreeRegressor(max_features=1.0, rando..."
AdaBoostRegressor,5715.740288,0.819971,"(DecisionTreeRegressor(max_depth=3, random_sta..."
RidgeCV,5797.596235,0.814778,"RidgeCV(alphas=[0.001, 0.01, 0.1, 1])"
Lasso,7290.125832,0.707135,"Lasso(alpha=16, random_state=100)"
LinearRegression,7628.690091,0.679301,LinearRegression()
ElasticNet,9278.926522,0.525547,ElasticNet(alpha=0.8)


# Testing for Better Accuracy

In [20]:
print(help(np.zeros))

Help on built-in function zeros in module numpy:

zeros(...)
    zeros(shape, dtype=float, order='C', *, like=None)
    
    Return a new array of given shape and type, filled with zeros.
    
    Parameters
    ----------
    shape : int or tuple of ints
        Shape of the new array, e.g., ``(2, 3)`` or ``2``.
    dtype : data-type, optional
        The desired data-type for the array, e.g., `numpy.int8`.  Default is
        `numpy.float64`.
    order : {'C', 'F'}, optional, default: 'C'
        Whether to store multi-dimensional data in row-major
        (C-style) or column-major (Fortran-style) order in
        memory.
    like : array_like
        Reference object to allow the creation of arrays which are not
        NumPy arrays. If an array-like passed in as ``like`` supports
        the ``__array_function__`` protocol, the result will be defined
        by it. In this case, it ensures the creation of an array object
        compatible with that passed in via this argument.
   

In [59]:
pid = 0

for k,v in product_dict.items():
  if k == "A9K1001-X":
    pid = v
    break

# Get all product data
df_Product = df_dp[df_dp['PLID'] == pid]
print(df_Product)
# Get Independent and Dependent
X = np.array(df_Product[['PLID','Quarter','Month','Year']])
y = np.array(df_Product['Booked_Qty'])

quarter_ohe = {'Q1':[1,0,0,0],'Q2':[0,1,0,0],'Q3':[0,0,1,0],'Q4':[0,0,0,1]}
month_ohe = {'JAN':[0,0,0,0,0,1,0,0,0,0,0,0],'FEB':[0,0,0,0,0,0,1,0,0,0,0,0],'MAR':[0,0,0,0,0,0,0,1,0,0,0,0],'APR':[0,0,0,0,0,0,0,0,1,0,0,0],
             'MAY':[0,0,0,0,0,0,0,0,0,1,0,0],'JUN':[0,0,0,0,0,0,0,0,0,0,1,0],'JUL':[0,0,0,0,0,0,0,0,0,0,0,1],'AUG':[1,0,0,0,0,0,0,0,0,0,0,0],
             'SEP':[0,1,0,0,0,0,0,0,0,0,0,0],'OCT':[0,0,1,0,0,0,0,0,0,0,0,0],'NOV':[0,0,0,1,0,0,0,0,0,0,0,0],'DEC':[0,0,0,0,1,0,0,0,0,0,0,0]}


Xen = []
for row in X:
    upq = quarter_ohe[f'{row[1]}']
    upm = month_ohe[f'{row[2]}']
    encodings = (upm, upq)
    Xen.append(upm+upq+[row[0]]+[row[3]])#encodings, [row[0]], [row[3]])

Xen = np.array(Xen)

print(Xen)

def foo(p, q, m, y):
  upq = quarter_ohe[f'{q}']
  upm = month_ohe[f'{m}']
  encodings = (upm, upq)
  return(upm+upq+[p]+[y])

# # One Hot Encoding Quarters
# ct_qt = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
# X = np.array(ct_qt.fit_transform(X))
# # One Hot Encoding Months
# ct_mon = ColumnTransformer(transformers=[('encoder', ohe, [-2])], remainder='passthrough')
# X = np.array(ct_mon.fit_transform(X))

# quarter_ohe = {}
# month_ohe = {}



       PLID  Booked_Qty Quarter Month  Year
52       23           0      Q3   MAR  2014
76       23          29      Q4   MAY  2014
93       23          15      Q4   JUN  2014
110      23         563      Q4   JUL  2014
162      23         921      Q2   JAN  2015
...     ...         ...     ...   ...   ...
9906     23         623      Q1   AUG  2022
10154    23         880      Q1   SEP  2022
10404    23        2075      Q1   OCT  2022
10653    23         795      Q2   NOV  2022
10903    23         669      Q2   DEC  2022

[94 rows x 5 columns]
[[   0    0    0 ...    0   23 2014]
 [   0    0    0 ...    1   23 2014]
 [   0    0    0 ...    1   23 2014]
 ...
 [   0    0    1 ...    0   23 2022]
 [   0    0    0 ...    0   23 2022]
 [   0    0    0 ...    0   23 2022]]


In [60]:
test = foo(pid, 'Q2', 'JAN', 2023)
print(y[0])
print(test)

0
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 23, 2023]


In [ ]:
print(h)

In [61]:
#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(Xen, y, test_size = 0.2, random_state = 42)

classifiers = [
    LinearRegression(),
    RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1]),
    Lasso(alpha =16, random_state=100),
    ElasticNet(alpha=0.8),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    GradientBoostingRegressor(),
    AdaBoostRegressor(),
    SVR(), 
    LinearSVR(), 
    xgb.XGBRegressor(),
    lgb.LGBMRegressor()
    ]

name = []
score = []
models = []
model_acc = {}
rmse = []
i = 0

for classifier in classifiers:
    classifier.fit(X_train, y_train)   
    name.append(type(classifier).__name__)
    score.append(classifier.score(X_test, y_test))
    models.append(classifier)
    rmse.append(np.sqrt(mean_squared_error(classifier.predict(X_test), y_test)))
    y_pred = classifier.predict([test])
    model_acc[type(classifier).__name__] = y_pred
    # break


df_score = pd.DataFrame(list(zip(name,rmse, score, models)),columns=['name','rmse','score',"model"])
df_score.set_index('name',inplace=True)
df_score.sort_values(by=['score'], ascending= False,inplace=True)
print(df_score)
print("&"*100)
print(model_acc)
print("&"*100)
print(y)

/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


                                  rmse     score  \
name                                               
GradientBoostingRegressor   330.340595  0.860391   
RandomForestRegressor       359.805721  0.834375   
XGBRegressor                393.790825  0.801610   
DecisionTreeRegressor       449.479524  0.741531   
AdaBoostRegressor           498.988940  0.681455   
LinearRegression            667.709987  0.429619   
RidgeCV                     685.377335  0.399036   
Lasso                       714.292082  0.347259   
ElasticNet                  834.177852  0.109761   
LGBMRegressor               863.662551  0.045716   
SVR                         888.342335 -0.009602   
LinearSVR                  1240.254372 -0.967935   

                                                                       model  
name                                                                          
GradientBoostingRegressor  ([DecisionTreeRegressor(criterion='friedman_ms...  
RandomForestRegressor      (Decisi

In [ ]:
print(df_score.iloc[0])

rmse                                           4323.819453
score                                             0.896977
model    (DecisionTreeRegressor(max_depth=3, random_sta...
Name: AdaBoostRegressor, dtype: object
